# Build your own neural network - solution

Now it is all up to you to build your own neural network from scratch and apply a hyperparameter search.

## The data

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('cars.csv')
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,doors,bodystyle,drive,enginelocation,wheelbase,...,enginesize,fuelsystem,bore,stroke,compression,hp,rpm,citympg,highwaympg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


In [2]:
df = df.fillna(0)

y = df[['price']]
X = df.drop(['price'],axis=1)

In [12]:
for column in X.columns:
    if X[column].dtype == np.object:
        print('Converting ', column)
        X = pd.concat([X,pd.get_dummies(X[column], prefix=column, drop_first=True)],axis=1).drop([column],axis=1)

/tmp/ipykernel_4346/2584167763.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if X[column].dtype == np.object:


## Create the model

Creating train and test set and standardising:

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

X_train = StandardScaler().fit_transform(X_train)
y_train = StandardScaler().fit_transform(y_train)

X_test = StandardScaler().fit(X_train).transform(X_test)
y_test = StandardScaler().fit(y_train).transform(y_test)

/opt/conda/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


### Baseline models: linear regression and random forest

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train,np.ravel(y_train))
prediction = lr.predict(X_test)

print('RMSE linear regression:', np.sqrt(mse(y_test,prediction)))

rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train,np.ravel(y_train))
prediction = rf.predict(X_test)

print('RMSE random forest:', np.sqrt(mse(y_test,prediction)))

RMSE linear regression: 12543.355631769884
RMSE random forest: 14441.677829374104


### Create your neural network

Incorporate the following:
- 2 different kernels in hidden layer.
- 2 different output kernels.
- Different sizes for the hidden layers.
- Different number of hidden layers.
- Use 10 epochs.

Here, create your model compatible with Keras:

In [14]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import Adam

def nn_model(no_neurons,no_layers,kernel,output_kernel):
    model = Sequential()
    model.add(Dense(no_neurons,input_dim=input_dim))
    model.add(Activation(kernel))

    # Extra hidden layers
    for _ in range(0,no_layers):
        model.add(Dense(no_neurons))
        model.add(Activation(kernel))

    # Output
    model.add(Dense(output_dim))
    model.add(Activation(output_kernel))
    model.compile(optimizer=Adam(),loss='mean_squared_error',metrics=['mean_squared_error'])
        
    return model

Here, run your grid search (10 epochs):

In [15]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

input_dim = X.shape[1]
output_dim = 1

parameters = {'no_neurons':[50,100],'kernel':['relu','linear'],'output_kernel':['linear','sigmoid'],'no_layers':[1,2],'verbose':[0]} 

grid_search = GridSearchCV(KerasRegressor(nn_model), parameters, cv=5,scoring='neg_mean_squared_error')
grid_search.fit(X, y,epochs=10)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']

print('Mean RMSE (+/- standard deviation), for parameters')
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/- %0.03f) for %r"
          # The MSE is return as a negative, so we multiple it with -1 before squaring it
          % (np.sqrt(-1*mean), np.sqrt(std), params))

/tmp/ipykernel_4346/3732959856.py:9: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  grid_search = GridSearchCV(KerasRegressor(nn_model), parameters, cv=5,scoring='neg_mean_squared_error')


Mean RMSE (+/- standard deviation), for parameters
9463.190 (+/- 8223.373) for {'kernel': 'relu', 'no_layers': 1, 'no_neurons': 50, 'output_kernel': 'linear', 'verbose': 0}
15252.193 (+/- 9343.586) for {'kernel': 'relu', 'no_layers': 1, 'no_neurons': 50, 'output_kernel': 'sigmoid', 'verbose': 0}
7779.395 (+/- 6764.810) for {'kernel': 'relu', 'no_layers': 1, 'no_neurons': 100, 'output_kernel': 'linear', 'verbose': 0}
15252.224 (+/- 9344.488) for {'kernel': 'relu', 'no_layers': 1, 'no_neurons': 100, 'output_kernel': 'sigmoid', 'verbose': 0}
7927.316 (+/- 6599.193) for {'kernel': 'relu', 'no_layers': 2, 'no_neurons': 50, 'output_kernel': 'linear', 'verbose': 0}
15252.019 (+/- 9343.650) for {'kernel': 'relu', 'no_layers': 2, 'no_neurons': 50, 'output_kernel': 'sigmoid', 'verbose': 0}
7636.916 (+/- 7068.432) for {'kernel': 'relu', 'no_layers': 2, 'no_neurons': 100, 'output_kernel': 'linear', 'verbose': 0}
15251.700 (+/- 9343.927) for {'kernel': 'relu', 'no_layers': 2, 'no_neurons': 100, 'ou

It seems that there is quite some difference between using the sigmiod and linear output kernel. The former gives a lower RMSE. The better results are obtained using ReLU as the activation function in the hidden layers.

Smaller hidden layers:

In [8]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

input_dim = X.shape[1]
output_dim = 1

parameters = {'no_neurons':[10,20],'kernel':['relu','linear'],'output_kernel':['linear','sigmoid'],'no_layers':[0,1],'verbose':[0]} 

grid_search = GridSearchCV(KerasClassifier(nn_model), parameters, cv=5,scoring='neg_mean_squared_error')
grid_search.fit(X, y,epochs=10)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']

print('Mean RMSE (+/- standard deviation), for parameters')
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/- %0.03f) for %r"
          # The MSE is return as a negative, so we multiple it with -1 before squaring it
          % (np.sqrt(-1*mean), np.sqrt(std), params))

/tmp/ipykernel_4346/4183698539.py:9: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  grid_search = GridSearchCV(KerasClassifier(nn_model), parameters, cv=5,scoring='neg_mean_squared_error')


2/2 [==============================] - 0s 3ms/step
Mean RMSE (+/- standard deviation), for parameters
12576.639 (+/- 7542.075) for {'kernel': 'relu', 'no_layers': 0, 'no_neurons': 10, 'output_kernel': 'linear', 'verbose': 0}
13552.264 (+/- 6462.121) for {'kernel': 'relu', 'no_layers': 0, 'no_neurons': 10, 'output_kernel': 'sigmoid', 'verbose': 0}
12037.993 (+/- 10248.099) for {'kernel': 'relu', 'no_layers': 0, 'no_neurons': 20, 'output_kernel': 'linear', 'verbose': 0}
13901.632 (+/- 11068.992) for {'kernel': 'relu', 'no_layers': 0, 'no_neurons': 20, 'output_kernel': 'sigmoid', 'verbose': 0}
11295.675 (+/- 8387.695) for {'kernel': 'relu', 'no_layers': 1, 'no_neurons': 10, 'output_kernel': 'linear', 'verbose': 0}
12866.716 (+/- 8107.994) for {'kernel': 'relu', 'no_layers': 1, 'no_neurons': 10, 'output_kernel': 'sigmoid', 'verbose': 0}
11327.412 (+/- 8461.682) for {'kernel': 'relu', 'no_layers': 1, 'no_neurons': 20, 'output_kernel': 'linear', 'verbose': 0}
11233.957 (+/- 8431.279) for {'k

We clearly have a higher error rate, meaning that a bigger network pays off in this case.

Let's try more epochs for the smaller hidden layers:

In [9]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

input_dim = X.shape[1]
output_dim = 1

parameters = {'no_neurons':[10,20],'kernel':['relu','linear'],'output_kernel':['linear','sigmoid'],'no_layers':[0,1],'verbose':[0]} 

grid_search = GridSearchCV(KerasClassifier(nn_model), parameters, cv=5,scoring='neg_mean_squared_error')
grid_search.fit(X, y,epochs=100)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']

print('Mean RMSE (+/- standard deviation), for parameters')
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/- %0.03f) for %r"
          # The MSE is return as a negative, so we multiple it with -1 before squaring it
          % (np.sqrt(-1*mean), np.sqrt(std), params))

/tmp/ipykernel_4346/3544118226.py:9: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  grid_search = GridSearchCV(KerasClassifier(nn_model), parameters, cv=5,scoring='neg_mean_squared_error')


2/2 [==============================] - 0s 2ms/step
Mean RMSE (+/- standard deviation), for parameters
11233.957 (+/- 8431.279) for {'kernel': 'relu', 'no_layers': 0, 'no_neurons': 10, 'output_kernel': 'linear', 'verbose': 0}
12011.618 (+/- 8522.002) for {'kernel': 'relu', 'no_layers': 0, 'no_neurons': 10, 'output_kernel': 'sigmoid', 'verbose': 0}
11233.957 (+/- 8431.279) for {'kernel': 'relu', 'no_layers': 0, 'no_neurons': 20, 'output_kernel': 'linear', 'verbose': 0}
12997.325 (+/- 8931.315) for {'kernel': 'relu', 'no_layers': 0, 'no_neurons': 20, 'output_kernel': 'sigmoid', 'verbose': 0}
12011.618 (+/- 8522.002) for {'kernel': 'relu', 'no_layers': 1, 'no_neurons': 10, 'output_kernel': 'linear', 'verbose': 0}
12743.251 (+/- 7673.428) for {'kernel': 'relu', 'no_layers': 1, 'no_neurons': 10, 'output_kernel': 'sigmoid', 'verbose': 0}
11264.247 (+/- 8439.524) for {'kernel': 'relu', 'no_layers': 1, 'no_neurons': 20, 'output_kernel': 'linear', 'verbose': 0}
12743.251 (+/- 7673.428) for {'ker

No difference here except for the reLU kernel based ones. we might be overfitting in the other cases.